# Text2SQL Module - T5 Small

In [ ]:
# Step 2: Install dependencies (run in a Colab cell if needed)
!pip install --upgrade datasets transformers tensorflow evaluate sqlalchemy sacrebleu rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 45.1 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=43cc6fda1acfa5d157cb1888b9a81aafc6899a39e815083eba3205244c74a7a2
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225

In [ ]:
import tensorflow as tf
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
from datasets import load_dataset
import sqlite3
import json
import evaluate
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, text
import os
from google.colab import drive

In [ ]:
# Step 1: Mount Google Drive (optional, for JSONL fallback or saving model)
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Step 3: Clear cache (optional, to avoid corrupted cache)
!rm -rf ~/.cache/huggingface/datasets

## Loading and Merging Data

In [ ]:
# Load wikisql dataset
data_dir='/content/drive/MyDrive/wikisql_data'

train_jsonl = os.path.join(data_dir, 'train.jsonl')
dev_jsonl = os.path.join(data_dir, 'dev.jsonl')
test_jsonl = os.path.join(data_dir, 'test.jsonl')
train_tables_jsonl = os.path.join(data_dir, 'train.tables.jsonl')
dev_tables_jsonl = os.path.join(data_dir, 'dev.tables.jsonl')
test_tables_jsonl = os.path.join(data_dir, 'test.tables.jsonl')

In [ ]:
# Check for file availability
for file_path in [train_jsonl, dev_jsonl, test_jsonl, train_tables_jsonl, dev_tables_jsonl, test_tables_jsonl]:
  if not os.path.exists(file_path):
    raise FileNotFoundError(f"File not found: {file_path}")

In [ ]:
# Helper Function: Load tables into DataFrames
def load_tables_jsonl(file_path):
  tables = []
  with open(file_path, 'r') as f:
    for line in f:
      table = json.loads(line.strip())
      tables.append({
        'id': table['id'],
        'header': table['header']
      })
  return pd.DataFrame(tables)

In [ ]:
train_tables_df = load_tables_jsonl(train_tables_jsonl)
dev_tables_df = load_tables_jsonl(dev_tables_jsonl)
test_tables_df = load_tables_jsonl(test_tables_jsonl)

NameError: name 'load_tables_jsonl' is not defined

In [ ]:
train_tables_df.head()

,id,header
0,1-1000181-1,"[State/territory, Text/background colour, Form..."
1,1-10006830-1,"[Aircraft, Description, Max Gross Weight, Tota..."
2,1-10007452-3,"[Order Year, Manufacturer, Model, Fleet Series..."
3,1-10015132-1,"[Player, No., Nationality, Position, Years in ..."
4,1-10015132-14,"[Player, No., Nationality, Position, Years in ..."


In [ ]:
# Helper Function: Load queries into DataFrames
def load_queries_jsonl(file_path, phase):
  queries = []
  with open(file_path, 'r') as f:
    for line in f:
      item = json.loads(line.strip())
      queries.append({
        'phase': phase,
        'question': item['question'],
        'table_id': item['table_id'],
        'sql': item['sql']
      })
  return pd.DataFrame(queries)

In [ ]:
train_queries_df = load_queries_jsonl(train_jsonl, 'train')
dev_queries_df = load_queries_jsonl(dev_jsonl, 'dev')
test_queries_df = load_queries_jsonl(test_jsonl, 'test')

In [ ]:
train_queries_df.head()

,phase,question,table_id,sql
0,train,Tell me what the notes are for South Australia,1-1000181-1,"{'sel': 5, 'conds': [[3, 0, 'SOUTH AUSTRALIA']..."
1,train,What is the current series where the new serie...,1-1000181-1,"{'sel': 4, 'conds': [[5, 0, 'New series began ..."
2,train,What is the format for South Australia?,1-1000181-1,"{'sel': 2, 'conds': [[0, 0, 'South Australia']..."
3,train,Name the background colour for the Australian ...,1-1000181-1,"{'sel': 1, 'conds': [[0, 0, 'Australian Capita..."
4,train,how many times is the fuel propulsion is cng?,1-10007452-3,"{'sel': 3, 'conds': [[5, 0, 'CNG']], 'agg': 3}"


In [ ]:
# Block 5: Merge queries with tables
def merge_queries_tables(queries_df, tables_df):
  table_dict = tables_df.set_index('id')[['header']].to_dict('index')
  queries_df['table'] = queries_df['table_id'].apply(
    lambda tid: {'id': tid, 'header': table_dict.get(tid, {}).get('header', [])}
  )
  return queries_df[['phase', 'question', 'table', 'sql']]

In [ ]:
train_df = merge_queries_tables(train_queries_df, train_tables_df)
dev_df = merge_queries_tables(dev_queries_df, dev_tables_df)
test_df = merge_queries_tables(test_queries_df, test_tables_df)

In [ ]:
# Check raw data
print("Train NaN questions:", train_df['question'].isna().sum())
print("Dev NaN questions:", dev_df['question'].isna().sum())
print("Sample train row:", train_df.iloc[0].to_dict())

Train NaN questions: 0
Dev NaN questions: 0
Sample train row: {'phase': 'train', 'question': 'Tell me what the notes are for South Australia ', 'table': {'id': '1-1000181-1', 'header': ['State/territory', 'Text/background colour', 'Format', 'Current slogan', 'Current series', 'Notes']}, 'sql': {'sel': 5, 'conds': [[3, 0, 'SOUTH AUSTRALIA']], 'agg': 0}}


In [ ]:
train_df.head()

,phase,question,table,sql
0,train,Tell me what the notes are for South Australia,"{'id': '1-1000181-1', 'header': ['State/territ...","{'sel': 5, 'conds': [[3, 0, 'SOUTH AUSTRALIA']..."
1,train,What is the current series where the new serie...,"{'id': '1-1000181-1', 'header': ['State/territ...","{'sel': 4, 'conds': [[5, 0, 'New series began ..."
2,train,What is the format for South Australia?,"{'id': '1-1000181-1', 'header': ['State/territ...","{'sel': 2, 'conds': [[0, 0, 'South Australia']..."
3,train,Name the background colour for the Australian ...,"{'id': '1-1000181-1', 'header': ['State/territ...","{'sel': 1, 'conds': [[0, 0, 'Australian Capita..."
4,train,how many times is the fuel propulsion is cng?,"{'id': '1-10007452-3', 'header': ['Order Year'...","{'sel': 3, 'conds': [[5, 0, 'CNG']], 'agg': 3}"


In [ ]:
# Block 6: Verify DataFrames
print("Train DataFrame:")
print(train_df.info())
print(f"Rows: {len(train_df)}")
print("\nValidation DataFrame:")
print(dev_df.info())
print(f"Rows: {len(dev_df)}")
print("\nTest DataFrame:")
print(test_df.info())
print(f"Rows: {len(test_df)}")

Train DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56355 entries, 0 to 56354
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   phase     56355 non-null  object
 1   question  56355 non-null  object
 2   table     56355 non-null  object
 3   sql       56355 non-null  object
dtypes: object(4)
memory usage: 1.7+ MB
None
Rows: 56355

Validation DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8421 entries, 0 to 8420
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   phase     8421 non-null   object
 1   question  8421 non-null   object
 2   table     8421 non-null   object
 3   sql       8421 non-null   object
dtypes: object(4)
memory usage: 263.3+ KB
None
Rows: 8421

Test DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15878 entries, 0 to 15877
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------ 

In [ ]:
# Convert to CSV format
train_df.to_csv(os.path.join(data_dir, 'merged_data/merged_train.csv'), index=False)
dev_df.to_csv(os.path.join(data_dir, 'merged_data/merged_dev.csv'), index=False)
test_df.to_csv(os.path.join(data_dir, 'merged_data/merged_test.csv'), index=False)

NameError: name 'train_df' is not defined

## Data Preprocess

**Input:** "Question: ..... | Table: .....(....*Column_names*....)"; string / text data

**Output:** SQL Query; string / text data

### Train Data

In [ ]:
# train_df = pd.read_csv(os.path.join(data_dir, "merged_data/merged_train.csv"))

In [ ]:
# Test by 1 record
# DP1: Extract input components
example = train_df.iloc[50,]

question = example['question']
table = example['table']
table_id = table['id']
columns = table['header']

print("question:", question)
print("table_id:", table_id)
print("columns:", columns)

question: what's the singles w-l for kim doo-hwan
table_id: 1-10023387-1
columns: ['Player', 'Years Played', 'Total W-L', 'Singles W-L', 'Doubles W-L']


In [ ]:
# DP2: Format input string
input_text = f"Question: {question} | Table: {table_id} ({', '.join(columns)})"
print(input_text)

Question: what's the singles w-l for kim doo-hwan | Table: 1-10023387-1 (Player, Years Played, Total W-L, Singles W-L, Doubles W-L)


In [ ]:
# agg_ops = ['', 'MAX', 'MIN', 'COUNT', 'SUM', 'AVG']
# cond_ops = ['=', '>', '<', 'OP']
# syms = ['SELECT', 'WHERE', 'AND', 'COL', 'TABLE', 'CAPTION', 'PAGE', 'SECTION', 'OP', 'COND', 'QUESTION', 'AGG', 'AGGOPS', 'CONDOPS']

In [ ]:
# DP3: Extract SQL components
aggs = {0: '', 1: 'MAX', 2: 'MIN', 3: 'COUNT', 4: 'SUM', 5: 'AVG'}

sql = example['sql']
sel_col = columns[sql['sel']]
agg = aggs[sql['agg']]
agg_str = f"{agg}({sel_col})" if agg else sel_col

print("SQL:", agg_str)

SQL: Singles W-L


In [ ]:
# DP 4: Process conditions
conds = []
ops = {0: '=', 1: '>', 2: '<', 3: '!='}

for cond in sql['conds']:
  col_idx, op_idx, val = cond
  op = ops[op_idx]
  val_str = f"'{val}'" if isinstance(val, str) and not val.isdigit() else str(val)
  conds.append(f"{columns[col_idx]} {op} {val_str}")

cond_str = ' AND '.join(conds) if conds else ''
where_clause = f" WHERE {cond_str}" if cond_str else ''

print(where_clause)


 WHERE Player = 'Kim Doo-Hwan'


In [ ]:
# DP 5: Construct output SQL
output_text = f"SELECT {agg_str} FROM {table_id}{where_clause};"
print(output_text)

SELECT Singles W-L FROM 1-10023387-1 WHERE Player = 'Kim Doo-Hwan';


In [ ]:
# Wrap into a function - process for each record
def preprocess_wikisql(example):

  # DP1: Extract input components
  question = example['question']
  table = example['table']
  table_id = table['id']
  columns = table['header']

  # DP2: Format input string
  input_text = f"Question: {question} | Table: {table_id} ({', '.join(columns)})"

  # DP3: Extract SQL components
  aggs = {0: '', 1: 'MAX', 2: 'MIN', 3: 'COUNT', 4: 'SUM', 5: 'AVG'}

  sql = example['sql']
  sel_col = columns[sql['sel']]
  agg = aggs[sql['agg']]
  agg_str = f"{agg}({sel_col})" if agg else sel_col

  # DP 4: Process conditions
  conds = []
  ops = {0: '=', 1: '>', 2: '<', 3: '!='}

  for cond in sql['conds']:
    col_idx, op_idx, val = cond
    op = ops[op_idx]
    val_str = f"'{val}'" if isinstance(val, str) and not val.isdigit() else str(val)
    conds.append(f"{columns[col_idx]} {op} {val_str}")

  cond_str = ' AND '.join(conds) if conds else ''
  where_clause = f" WHERE {cond_str}" if cond_str else ''

  # DP 5: Construct output SQL
  output_text = f"SELECT {agg_str} FROM {table_id}{where_clause};"

  # Return in dict
  return {
    'input_text': input_text.replace('\xa0', ' '), # standardise for space symbol
    'output_text': output_text.replace('\xa0', ' '), # standardise for space symbol
    'table_id': table_id
  }


In [ ]:
# Test with example 2
example_2 = train_df.iloc[100,]
result = preprocess_wikisql(example_2)
print(result)

{'input_text': 'Question: What is the season year where the rank is 39? | Table: 1-10120207-8 (Season, Timeslot ( ET ), Season premiere, Season finale, TV season, Rank, Viewers (millions))', 'output_text': 'SELECT TV season FROM 1-10120207-8 WHERE Rank = 39;', 'table_id': '1-10120207-8'}


In [ ]:
# Preprocess data
train_data_processed = train_df.apply(preprocess_wikisql, axis=1).tolist()
print(f"\nProcessed Rows: {len(train_data_processed)}")
print("\nFirst 3 Preprocessed Examples:")
for i in range(3):
    print(train_data_processed[i])


Processed Rows: 56355

First 3 Preprocessed Examples:
{'input_text': 'Question: Tell me what the notes are for South Australia  | Table: 1-1000181-1 (State/territory, Text/background colour, Format, Current slogan, Current series, Notes)', 'output_text': "SELECT Notes FROM 1-1000181-1 WHERE Current slogan = 'SOUTH AUSTRALIA';", 'table_id': '1-1000181-1'}
{'input_text': 'Question: What is the current series where the new series began in June 2011? | Table: 1-1000181-1 (State/territory, Text/background colour, Format, Current slogan, Current series, Notes)', 'output_text': "SELECT Current series FROM 1-1000181-1 WHERE Notes = 'New series began in June 2011';", 'table_id': '1-1000181-1'}
{'input_text': 'Question: What is the format for South Australia? | Table: 1-1000181-1 (State/territory, Text/background colour, Format, Current slogan, Current series, Notes)', 'output_text': "SELECT Format FROM 1-1000181-1 WHERE State/territory = 'South Australia';", 'table_id': '1-1000181-1'}


In [ ]:
# Save train data to jsonl
output_path = '/content/drive/MyDrive/wikisql_data/preprocessed_data/train_data_processed.jsonl'
with open(output_path, 'w') as f:
    for item in train_data_processed:
        f.write(json.dumps(item) + '\n')
print(f"\nPreprocessed data saved to: {output_path}")


Preprocessed data saved to: /content/drive/MyDrive/wikisql_data/preprocessed_data/train_data_processed.jsonl


In [ ]:
# Do the same for dev dataset
dev_data_processed = dev_df.apply(preprocess_wikisql, axis=1).tolist()
print(f"\nProcessed Rows: {len(dev_data_processed)}")
print("\nFirst 3 Preprocessed Examples:")
for i in range(3):
    print(dev_data_processed[i])


Processed Rows: 8421

First 3 Preprocessed Examples:
{'input_text': 'Question: What position does the player who played for butler cc (ks) play? | Table: 1-10015132-11 (Player, No., Nationality, Position, Years in Toronto, School/Club Team)', 'output_text': "SELECT Position FROM 1-10015132-11 WHERE School/Club Team = 'Butler CC (KS)';", 'table_id': '1-10015132-11'}
{'input_text': 'Question: How many schools did player number 3 play at? | Table: 1-10015132-11 (Player, No., Nationality, Position, Years in Toronto, School/Club Team)', 'output_text': 'SELECT COUNT(School/Club Team) FROM 1-10015132-11 WHERE No. = 3;', 'table_id': '1-10015132-11'}
{'input_text': 'Question: What school did player number 21 play for? | Table: 1-10015132-11 (Player, No., Nationality, Position, Years in Toronto, School/Club Team)', 'output_text': 'SELECT School/Club Team FROM 1-10015132-11 WHERE No. = 21;', 'table_id': '1-10015132-11'}


In [ ]:
# Save dev data to jsonl
output_path = '/content/drive/MyDrive/wikisql_data/preprocessed_data/dev_data_processed.jsonl'
with open(output_path, 'w') as f:
    for item in dev_data_processed:
        f.write(json.dumps(item) + '\n')
print(f"\nPreprocessed data saved to: {output_path}")


Preprocessed data saved to: /content/drive/MyDrive/wikisql_data/preprocessed_data/dev_data_processed.jsonl


In [ ]:
# Extract the header columns for each record
train_df.head()

,phase,question,table,sql
0,train,Tell me what the notes are for South Australia,"{'id': '1-1000181-1', 'header': ['State/territ...","{'sel': 5, 'conds': [[3, 0, 'SOUTH AUSTRALIA']..."
1,train,What is the current series where the new serie...,"{'id': '1-1000181-1', 'header': ['State/territ...","{'sel': 4, 'conds': [[5, 0, 'New series began ..."
2,train,What is the format for South Australia?,"{'id': '1-1000181-1', 'header': ['State/territ...","{'sel': 2, 'conds': [[0, 0, 'South Australia']..."
3,train,Name the background colour for the Australian ...,"{'id': '1-1000181-1', 'header': ['State/territ...","{'sel': 1, 'conds': [[0, 0, 'Australian Capita..."
4,train,how many times is the fuel propulsion is cng?,"{'id': '1-10007452-3', 'header': ['Order Year'...","{'sel': 3, 'conds': [[5, 0, 'CNG']], 'agg': 3}"


In [ ]:
def extract_columns(example):
  return example['table']['header']

print(extract_columns(example))

['Player', 'Years Played', 'Total W-L', 'Singles W-L', 'Doubles W-L']


In [ ]:
train_columns = train_df.apply(extract_columns, axis=1).to_list()
for col in train_columns[:10]:
  print(col)

['State/territory', 'Text/background colour', 'Format', 'Current slogan', 'Current series', 'Notes']
['State/territory', 'Text/background colour', 'Format', 'Current slogan', 'Current series', 'Notes']
['State/territory', 'Text/background colour', 'Format', 'Current slogan', 'Current series', 'Notes']
['State/territory', 'Text/background colour', 'Format', 'Current slogan', 'Current series', 'Notes']
['Order Year', 'Manufacturer', 'Model', 'Fleet Series (Quantity)', 'Powertrain (Engine/Transmission)', 'Fuel Propulsion']
['Order Year', 'Manufacturer', 'Model', 'Fleet Series (Quantity)', 'Powertrain (Engine/Transmission)', 'Fuel Propulsion']
['Order Year', 'Manufacturer', 'Model', 'Fleet Series (Quantity)', 'Powertrain (Engine/Transmission)', 'Fuel Propulsion']
['Order Year', 'Manufacturer', 'Model', 'Fleet Series (Quantity)', 'Powertrain (Engine/Transmission)', 'Fuel Propulsion']
['Order Year', 'Manufacturer', 'Model', 'Fleet Series (Quantity)', 'Powertrain (Engine/Transmission)', 'Fuel

In [ ]:
# Save train columns to jsonl
output_path = '/content/drive/MyDrive/wikisql_data/preprocessed_data/train_columns.jsonl'
with open(output_path, 'w') as f:
    for item in train_columns:
        f.write(json.dumps(item) + '\n')
print(f"\nColumns data saved to: {output_path}")


Columns data saved to: /content/drive/MyDrive/wikisql_data/preprocessed_data/train_columns.jsonl


In [ ]:
dev_columns = dev_df.apply(extract_columns, axis=1).to_list()
for col in dev_columns[:10]:
  print(col)

['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team']
['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team']
['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team']
['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team']
['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team']
['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team']
['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team']
['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team']
['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team']
['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team']


In [ ]:
# Save dev columns to jsonl
output_path = '/content/drive/MyDrive/wikisql_data/preprocessed_data/dev_columns.jsonl'
with open(output_path, 'w') as f:
    for item in dev_columns:
        f.write(json.dumps(item) + '\n')
print(f"\nColumns data saved to: {output_path}")


Columns data saved to: /content/drive/MyDrive/wikisql_data/preprocessed_data/dev_columns.jsonl


## Tokenize and Convert to TF Dataset for Model Training

In [ ]:
# Import data from saved processed data
import json

train_path = '/content/drive/MyDrive/wikisql_data/preprocessed_data/train_data_processed.jsonl'
dev_path = '/content/drive/MyDrive/wikisql_data/preprocessed_data/dev_data_processed.jsonl'

train_columns_path = '/content/drive/MyDrive/wikisql_data/preprocessed_data/train_columns.jsonl'
dev_columns_path = '/content/drive/MyDrive/wikisql_data/preprocessed_data/dev_columns.jsonl'

train_data_processed = []
with open(train_path, 'r') as f:
    for line in f:
        train_data_processed.append(json.loads(line))

print(f"\nLoaded {len(train_data_processed)} records from: {train_path}")

dev_data_processed = []
with open(dev_path, 'r') as f:
    for line in f:
        dev_data_processed.append(json.loads(line))

print(f"\nLoaded {len(dev_data_processed)} records from: {dev_path}")

train_columns = []
with open(train_columns_path, 'r') as f:
    for line in f:
        train_columns.append(json.loads(line))

print(f"\nLoaded {len(train_columns)} records from: {train_columns_path}")

dev_columns = []
with open(dev_columns_path, 'r') as f:
    for line in f:
        dev_columns.append(json.loads(line))

print(f"\nLoaded {len(dev_columns)} records from: {dev_columns_path}")


Loaded 56355 records from: /content/drive/MyDrive/wikisql_data/preprocessed_data/train_data_processed.jsonl

Loaded 8421 records from: /content/drive/MyDrive/wikisql_data/preprocessed_data/dev_data_processed.jsonl

Loaded 56355 records from: /content/drive/MyDrive/wikisql_data/preprocessed_data/train_columns.jsonl

Loaded 8421 records from: /content/drive/MyDrive/wikisql_data/preprocessed_data/dev_columns.jsonl


In [ ]:
# Check first 3 data from train and dev
print(f"\nProcessed Rows: {len(train_data_processed)}")
print("\nFirst 3 Preprocessed Examples:")
for i in range(3):
    print(train_data_processed[i])

print()

print(f"\nProcessed Rows: {len(dev_data_processed)}")
print("\nFirst 3 Preprocessed Examples:")
for i in range(3):
    print(dev_data_processed[i])


Processed Rows: 56355

First 3 Preprocessed Examples:
{'input_text': 'Question: Tell me what the notes are for South Australia  | Table: 1-1000181-1 (State/territory, Text/background colour, Format, Current slogan, Current series, Notes)', 'output_text': "SELECT Notes FROM 1-1000181-1 WHERE Current slogan = 'SOUTH AUSTRALIA';", 'table_id': '1-1000181-1'}
{'input_text': 'Question: What is the current series where the new series began in June 2011? | Table: 1-1000181-1 (State/territory, Text/background colour, Format, Current slogan, Current series, Notes)', 'output_text': "SELECT Current series FROM 1-1000181-1 WHERE Notes = 'New series began in June 2011';", 'table_id': '1-1000181-1'}
{'input_text': 'Question: What is the format for South Australia? | Table: 1-1000181-1 (State/territory, Text/background colour, Format, Current slogan, Current series, Notes)', 'output_text': "SELECT Format FROM 1-1000181-1 WHERE State/territory = 'South Australia';", 'table_id': '1-1000181-1'}


Proces

In [ ]:
# Check for empty examples
for i, item in enumerate(train_data_processed):
    if not item['input_text'] or not item['output_text']:
        print(f"Empty train item at index {i}:", item)
for i, item in enumerate(dev_data_processed):
    if not item['input_text'] or not item['output_text']:
        print(f"Empty dev item at index {i}:", item)

In [ ]:
# Convert to Hugging Face Dataset
from datasets import Dataset

train_dataset = Dataset.from_list(train_data_processed)
dev_dataset = Dataset.from_list(dev_data_processed)

### Tokenization

In [ ]:
# Initialize tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
# Try one example
example = train_dataset[0]

input = example['input_text']
target = example['output_text']

print("Input: ", input)
print("Target: ", target)

Input:  Question: Tell me what the notes are for South Australia  | Table: 1-1000181-1 (State/territory, Text/background colour, Format, Current slogan, Current series, Notes)
Target:  SELECT Notes FROM 1-1000181-1 WHERE Current slogan = 'SOUTH AUSTRALIA';


In [ ]:
max_length= 128

model_input = tokenizer(
  input,
  max_length=max_length,
  padding='max_length',
  truncation=True,
  # return_tensors='tf'
)

label = tokenizer(
  target,
  max_length=max_length,
  padding='max_length',
  truncation=True,
  # return_tensors='tf'
)

In [ ]:
# Check input_id, attention_mask, labels
print("Input ID:", len(model_input['input_ids']), model_input['input_ids'])
print("Attention mask:", len(model_input['attention_mask']), model_input['attention_mask'])
print("Labels:", len(label['input_ids']), label['input_ids'])

Input ID: 128 [11860, 10, 8779, 140, 125, 8, 3358, 33, 21, 1013, 2051, 1820, 4398, 10, 209, 4536, 17465, 4959, 2292, 41, 134, 4748, 87, 17, 21301, 10972, 6, 5027, 87, 1549, 9232, 3243, 6, 12439, 6, 12892, 22031, 6, 12892, 939, 6, 2507, 7, 61, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Attention mask: 128 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Labels: 128 [3, 23143, 14196, 2507, 7, 21680, 209, 4536, 17465, 4959, 2292, 549, 17444, 427, 12892, 

In [ ]:
# Wrap into function
def preprocess_function(example):
  input = example['input_text']
  target = example['output_text']

  max_length = 128

  model_input = tokenizer(
    input,
    max_length=max_length,
    padding='max_length',
    truncation=True,
    # return_tensors='tf'
  )

  label = tokenizer(
    target,
    max_length=max_length,
    padding='max_length',
    truncation=True,
    # return_tensors='tf'
  )

  return {
      'input_ids': model_input['input_ids'],
      'attention_mask': model_input['attention_mask'],
      'labels': label['input_ids']
  }

In [ ]:
# Test using 30% train data first
# sampled_train_data = train_dataset.train_test_split(test_size=0.7, shuffle=True, seed=42)['train']
# sampled_train_data.shape

In [ ]:
# Apply preprocessing
train_dataset = train_dataset.map(preprocess_function)
dev_dataset = dev_dataset.map(preprocess_function)

Map:   0%|          | 0/56355 [00:00<?, ? examples/s]

Map:   0%|          | 0/8421 [00:00<?, ? examples/s]

In [ ]:
print(train_dataset[0])

{'input_text': 'Question: Tell me what the notes are for South Australia  | Table: 1-1000181-1 (State/territory, Text/background colour, Format, Current slogan, Current series, Notes)', 'output_text': "SELECT Notes FROM 1-1000181-1 WHERE Current slogan = 'SOUTH AUSTRALIA';", 'table_id': '1-1000181-1', 'input_ids': [11860, 10, 8779, 140, 125, 8, 3358, 33, 21, 1013, 2051, 1820, 4398, 10, 209, 4536, 17465, 4959, 2292, 41, 134, 4748, 87, 17, 21301, 10972, 6, 5027, 87, 1549, 9232, 3243, 6, 12439, 6, 12892, 22031, 6, 12892, 939, 6, 2507, 7, 61, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
print(train_dataset[0]['input_ids'][:10], train_dataset[0]['labels'][:10])

[11860, 10, 8779, 140, 125, 8, 3358, 33, 21, 1013] [3, 23143, 14196, 2507, 7, 21680, 209, 4536, 17465, 4959]


In [ ]:
# Convert to TensorFlow dataset
batch_size = 16
tf_train_dataset = train_dataset.to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=lambda x: {
        "input_ids": tf.stack([tf.convert_to_tensor(item["input_ids"]) for item in x]),
        "attention_mask": tf.stack([tf.convert_to_tensor(item["attention_mask"]) for item in x]),
        "labels": tf.stack([tf.convert_to_tensor(item["labels"]) for item in x]),
    }
)

tf_dev_dataset = dev_dataset.to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=lambda x: {
        "input_ids": tf.stack([tf.convert_to_tensor(item["input_ids"]) for item in x]),
        "attention_mask": tf.stack([tf.convert_to_tensor(item["attention_mask"]) for item in x]),
        "labels": tf.stack([tf.convert_to_tensor(item["labels"]) for item in x]),
    }
)


In [ ]:
# Verify dataset
for i, batch in enumerate(tf_train_dataset.take(3)):
    print(f"Batch {i} shapes:", {k: v.shape for k, v in batch.items()})
    if tf.reduce_all(batch["input_ids"] == 0) or tf.reduce_all(batch["labels"] == 0):
        print(f"Invalid batch {i}: all-zero input_ids or labels")

Batch 0 shapes: {'input_ids': TensorShape([16, 128]), 'attention_mask': TensorShape([16, 128]), 'labels': TensorShape([16, 128])}
Batch 1 shapes: {'input_ids': TensorShape([16, 128]), 'attention_mask': TensorShape([16, 128]), 'labels': TensorShape([16, 128])}
Batch 2 shapes: {'input_ids': TensorShape([16, 128]), 'attention_mask': TensorShape([16, 128]), 'labels': TensorShape([16, 128])}


## T5 Small Transformer - Model Training

### Train wth T5 Small base model

In [ ]:
# Initialize model
model_name = "t5-small"
model = TFT5ForConditionalGeneration.from_pretrained(model_name, from_pt=True)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
# Zero Shot Inference
example = train_dataset[0]['input_text']

# Convert your lists to tensors
example_inputs = tokenizer(example, return_tensors='tf', max_length=128, padding=True, truncation=True)

outputs = model.generate(
    input_ids=example_inputs['input_ids'],
    attention_mask=example_inputs['attention_mask'],
    max_length=512
)

sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("NL Query: ", train_dataset[0]['input_text'])
print("GT SQL: ", train_dataset[0]['output_text'])
print("Pred SQL: ", sql_query)

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


NL Query:  Question: Tell me what the notes are for South Australia  | Table: 1-1000181-1 (State/territory, Text/background colour, Format, Current slogan, Current series, Notes)
GT SQL:  SELECT Notes FROM 1-1000181-1 WHERE Current slogan = 'SOUTH AUSTRALIA';
Pred SQL:  True


The base model can't predict any sql at all.

In [ ]:
# Compile model
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss)

In [ ]:
# Test forward pass
batch = next(iter(tf_train_dataset))
try:
    outputs = model(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        labels=batch["labels"],
        training=True
    )
    print("Logits shape:", outputs.logits.shape)
except Exception as e:
    print("Forward pass failed:", str(e))

Logits shape: (16, 128, 32128)


In [ ]:
batch = next(iter(tf_train_dataset))
print("Input IDs:", batch["input_ids"].numpy())
print("Labels:", batch["labels"].numpy())

Input IDs: [[11860    10   363 ...     0     0     0]
 [11860    10   363 ...     0     0     0]
 [11860    10   363 ...     0     0     0]
 ...
 [11860    10   363 ...     0     0     0]
 [11860    10   363 ...     0     0     0]
 [11860    10   571 ...     0     0     0]]
Labels: [[    3 23143 14196 ...     0     0     0]
 [    3 23143 14196 ...     0     0     0]
 [    3 23143 14196 ...     0     0     0]
 ...
 [    3 23143 14196 ...     0     0     0]
 [    3 23143 14196 ...     0     0     0]
 [    3 23143 14196 ...     0     0     0]]


In [ ]:
# Model training
history = model.fit(
    tf_train_dataset,
    validation_data=tf_dev_dataset,
    epochs=8,
)

Epoch 1/8
3523/3523 [==============================] - 1334s 359ms/step - loss: 0.2682 - val_loss: 0.0604
Epoch 2/8
3523/3523 [==============================] - 1272s 361ms/step - loss: 0.0731 - val_loss: 0.0400
Epoch 3/8
3523/3523 [==============================] - 1272s 361ms/step - loss: 0.0531 - val_loss: 0.0331
Epoch 4/8
3523/3523 [==============================] - 1272s 361ms/step - loss: 0.0440 - val_loss: 0.0295
Epoch 5/8
3523/3523 [==============================] - 1270s 361ms/step - loss: 0.0387 - val_loss: 0.0269
Epoch 6/8
3523/3523 [==============================] - 1272s 361ms/step - loss: 0.0350 - val_loss: 0.0253
Epoch 7/8
3523/3523 [==============================] - 1271s 361ms/step - loss: 0.0323 - val_loss: 0.0242
Epoch 8/8
3523/3523 [==============================] - 1270s 360ms/step - loss: 0.0300 - val_loss: 0.0233


In [ ]:
# Save model and tokenizer
model.save_pretrained('/content/drive/MyDrive/AP1/models/base_t5_small_text2sql_model')
tokenizer.save_pretrained('/content/drive/MyDrive/AP1/models/base_t5_small_text2sql_model')
print("Model and tokenizer saved to /content/drive/MyDrive/AP1/models/base_t5_small_text2sql_model.")

Model and tokenizer saved to /content/drive/MyDrive/AP1/models/base_t5_small_text2sql_model.


## Model Evaluation

In [ ]:
# try to load from files
model_path = '/content/drive/MyDrive/AP1/models/base_t5_small_text2sql_model'
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = TFT5ForConditionalGeneration.from_pretrained(model_path)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at /content/drive/MyDrive/AP1/models/base_t5_small_text2sql_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
num_examples = len(dev_data_processed)
batch_size = 16
num_batches = (num_examples + batch_size - 1) // batch_size

num_examples, batch_size, num_batches

(8421, 16, 527)

In [ ]:
# Validation data prediction
from tqdm import tqdm

predictions = []
with tqdm(total=num_batches, desc="Generating Predictions", unit="batch") as pbar:
    for batch in tf_dev_dataset:
        try:
            outputs = model.generate(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                max_length=max_length,
                num_beams=4,
                early_stopping=True
            )
            batch_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            predictions.extend(batch_preds)
            pbar.update(1)
        except Exception as e:
            print(f"Error in batch: {str(e)}")
            pbar.update(1)
print(f"\nGenerated {len(predictions)} predictions")

Generating Predictions: 100%|██████████| 527/527 [1:25:18<00:00,  9.71s/batch]


Generated 8421 predictions


In [ ]:
# Save predictions (in case runtime disconnected)
output_path = '/content/drive/MyDrive/AP1/predictions/base_dev_predictions.jsonl'
with open(output_path, 'w') as f:
    for item in predictions:
        f.write(json.dumps(item) + '\n')
print(f"\nPreprocessed data saved to: {output_path}")


Preprocessed data saved to: /content/drive/MyDrive/AP1/predictions/base_dev_predictions.jsonl


## Load the Prediction for Evaluation

In [ ]:
prediction_path = os.path.join('/content/drive/MyDrive/AP1/predictions', 'base_dev_predictions.jsonl')

predictions = []
with open(prediction_path, 'r') as f:
    for line in f:
        predictions.append(json.loads(line))

print(f"\nLoaded {len(predictions)} records from: {prediction_path}")
print("\nPrint first 5 predictions:")
for pred in predictions[:5]:
  print(pred)


Loaded 8421 records from: /content/drive/MyDrive/AP1/predictions/base_dev_predictions.jsonl

Print first 5 predictions:
SELECT Position FROM 1-10015132-11 WHERE School/Club Team = 'Butler CCC (Ks)';
SELECT COUNT(School/Club Team) FROM 1-10015132-11 WHERE No. = 3;
SELECT School/Club Team FROM 1-10015132-11 WHERE No. = 21;
SELECT Player FROM 1-10015132-11 WHERE No. = 42;
SELECT Player FROM 1-10015132-11 WHERE Position = 'Gardguard' AND Years in Toronto = '1996-97';


In [ ]:
# Evaluate 1: Exact Match Accuracy
def normalize_sql(query):
    return query.strip().rstrip(';').lower().replace('  ', ' ')

ground_truth = [normalize_sql(item['output_text']) for item in dev_data_processed]
normalized_predictions = [normalize_sql(pred) for pred in predictions]
exact_match_accuracy = sum(p == gt for p, gt in zip(normalized_predictions, ground_truth)) / len(ground_truth)
print(f"Exact Match Accuracy: {exact_match_accuracy:.4f}")

Exact Match Accuracy: 0.5360


- Exact Match Accuracy: 0.5360

In [ ]:
# Log mismatches (first 5)
mismatches = [(p, gt, item['input_text']) for p, gt, item in zip(normalized_predictions, ground_truth, dev_data_processed) if p != gt]
print(f"Number of Mismatches: {len(mismatches)}")
if mismatches[:5]:
    print("Sample Mismatches (Predicted, Ground Truth, Input):")
    for p, gt, inp in mismatches[:5]:
        print(f"Input: {inp}\nPred: {p}\nGT: {gt}\n")

Number of Mismatches: 3907
Sample Mismatches (Predicted, Ground Truth, Input):
Input: Question: What position does the player who played for butler cc (ks) play? | Table: 1-10015132-11 (Player, No., Nationality, Position, Years in Toronto, School/Club Team)
Pred: select position from 1-10015132-11 where school/club team = 'butler ccc (ks)'
GT: select position from 1-10015132-11 where school/club team = 'butler cc (ks)'

Input: Question: What player played guard for toronto in 1996-97? | Table: 1-10015132-11 (Player, No., Nationality, Position, Years in Toronto, School/Club Team)
Pred: select player from 1-10015132-11 where position = 'gardguard' and years in toronto = '1996-97'
GT: select player from 1-10015132-11 where position = 'guard' and years in toronto = '1996-97'

Input: Question: When did the Prime Minister of Italy take office? | Table: 1-10026563-1 (Entered office as Head of State or Government, Began time as senior G8 leader, Ended time as senior G8 leader, Person, Office)


 **Execution accuracy**:

 If want to do, need a series of NLP pipeline to change SQL query to wikisql database standard.

 1. **table_id:** 1-10026563-1 --> table_1_10026563_1
 2. **column name:** Position, Name... --> col1, col2...
 3. **lower case**


 Temporary skip this metrics

In [ ]:
print(dev_columns[:5])

[['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team'], ['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team'], ['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team'], ['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team'], ['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team']]


In [ ]:
import re

def extract_info(sql, columns):
  try:
    select_pattern = r"SELECT\s+(.*?)\s+FROM"
    select_match = re.search(select_pattern, sql, re.IGNORECASE)
    selected_cols_str = select_match.group(1)
    raw_selected_columns = [col.strip() for col in selected_cols_str.split(",")]

    # 2. Map each selected column (handling aggregates)
    mapped_columns = []
    agg_pattern = r"(\w+)\((.+)\)"  # e.g., COUNT(Player)

    for col in raw_selected_columns:
      agg_match = re.match(agg_pattern, col)
      if agg_match:
        func = agg_match.group(1)
        inner_col = agg_match.group(2).strip()
        if inner_col in columns:
          idx = columns.index(inner_col)
          mapped_columns.append(f"{func}(col{idx})")
        else:
          mapped_columns.append(col)  # fallback if column not found
      else:
        if col in columns:
          idx = columns.index(col)
          mapped_columns.append(f"col{idx}")
        else:
          mapped_columns.append(col)

    # 2. Extract table name
    table_pattern = r"FROM\s+([^\s;]+)"
    raw_table = re.search(table_pattern, sql, re.IGNORECASE).group(1)
    table = "table_" + raw_table.replace("-", "_")

    # 3. Extract where_clauses
    where_pattern = r"WHERE\s+(.*?);?$"
    where_match = re.search(where_pattern, sql, re.IGNORECASE)
    mapped_conditions = []

    if where_match:
      where_clause = where_match.group(1)

      # Split by AND / OR, preserving them
      tokens = re.split(r"(\s+AND\s+|\s+OR\s+)", where_clause)
      for token in tokens:
        token = token.strip()
        if token in ("AND", "OR"):
          mapped_conditions.append(token.upper())
        elif token:
          # Extract condition: column op value
          cond_pattern = r"(.+?)(=|!=|<|>)(.+)"
          cond_match = re.match(cond_pattern, token.strip())
          if cond_match:
            col, op, val = cond_match.group(1).strip(), cond_match.group(2), cond_match.group(3).strip()
            if col in columns:
              col_idx = columns.index(col)
              mapped_conditions.append(f"col{col_idx} {op} {val.lower()}")
            else:
              mapped_conditions.append(token)  # leave untouched if not found

    # Final result
    # print("Selected Columns:", mapped_columns)
    # print("Table:", table)
    # print("Where Clauses:", mapped_conditions)
    result = f"SELECT {', '.join(mapped_columns)} FROM {table}"
    if mapped_conditions:
      result += f" WHERE {' '.join(mapped_conditions)}"
    result += ";"
    return result

  except Exception as e:
    return ""


sql = "SELECT MIN(Population (Mareeba)) FROM 1-12555835-1 WHERE Population (Atherton) = 'Mareeba';"
selected_columns = dev_columns[248]

print(extract_info(sql, selected_columns))

SELECT MIN(col2) FROM table_1_12555835_1 WHERE col3 = 'mareeba';


In [ ]:
print(predictions[248])
dev_data_processed[248]['output_text']

SELECT MIN(Population (Mareeba)) FROM 1-12555835-1;


'SELECT MIN(Population (Mareeba)) FROM 1-12555835-1;'

In [ ]:
# Sample a prediction
for a, b in zip(predictions[:100], dev_columns[:100]):
  print(a)
  print(b)
  print(extract_info(a,b))

SELECT Position FROM 1-10015132-11 WHERE School/Club Team = 'Butler CCC (Ks)';
['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team']
SELECT col3 FROM table_1_10015132_11 WHERE col5 = 'butler ccc (ks)';
SELECT COUNT(School/Club Team) FROM 1-10015132-11 WHERE No. = 3;
['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team']
SELECT COUNT(col5) FROM table_1_10015132_11 WHERE col1 = 3;
SELECT School/Club Team FROM 1-10015132-11 WHERE No. = 21;
['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team']
SELECT col5 FROM table_1_10015132_11 WHERE col1 = 21;
SELECT Player FROM 1-10015132-11 WHERE No. = 42;
['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Club Team']
SELECT col0 FROM table_1_10015132_11 WHERE col1 = 42;
SELECT Player FROM 1-10015132-11 WHERE Position = 'Gardguard' AND Years in Toronto = '1996-97';
['Player', 'No.', 'Nationality', 'Position', 'Years in Toronto', 'School/Cl

In [ ]:
# Evaluate 2: Execution Accuracy
def execute_query(db_path, query, table_id):
  try:
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    conn.close()
    print("Execute successfully ...")
    return result
  except sqlite3.OperationalError as op_err:
    print(f"OperationalError: {op_err}\nQuery: {query}\nTable: {table_id}")
    conn.close()
    return None
  except sqlite3.DatabaseError as db_err:
    print(f"DatabaseError: {db_err}\nQuery: {query}\nTable: {table_id}")
    conn.close()
    return None
  except Exception as e:
    print(f"Unexpected Error: {str(e)}\nQuery: {query}\nTable: {table_id}")
    conn.close()
    return None

data_dir='/content/drive/MyDrive/wikisql_data'
dev_db = os.path.join(data_dir, 'dev.db')
query = "SELECT col3 FROM table_1_10015132_11 WHERE col5 = 'butler cc (ks)';"

execute_query(dev_db, query, "table_1_10015132_11")

Execute successfully ...


[('guard-forward',)]

In [ ]:
data_dir='/content/drive/MyDrive/wikisql_data'
dev_db = os.path.join(data_dir, 'dev.db')

correct = 0
failures = []
i = 0

for pred, item, cols in zip(predictions, dev_data_processed, dev_columns):
  pred_sql = extract_info(pred, cols)
  truth_sql = extract_info(item['output_text'], cols)

  print(i)
  print(item['output_text'])
  pred_result = execute_query(dev_db, pred_sql, item['table_id'])
  truth_result = execute_query(dev_db, truth_sql, item['table_id'])

  if truth_result == pred_result and truth_result is not None:
    correct += 1
  elif pred_result is None:
    failures.append((pred, item['output_text'], item['input_text']))

  i+=1

execution_accuracy = correct / len(dev_data_processed)
print(f"Execution Accuracy: {execution_accuracy:.4f}")
print(failures)

Streaming output truncated to the last 5000 lines.
7281
SELECT Result FROM 2-18444214-1 WHERE Date = 'february 27, 2000';
Execute successfully ...
Execute successfully ...
7282
SELECT Competition FROM 2-18444214-1 WHERE Score = '3–1';
Execute successfully ...
Execute successfully ...
7283
SELECT Score FROM 2-18444214-1 WHERE Competition = 'king's cup 2000';
OperationalError: near "s": syntax error
Query: SELECT col2 FROM table_2_18444214_1 WHERE col4 = 'king's cup 2000';
Table: 2-18444214-1
OperationalError: near "s": syntax error
Query: SELECT col2 FROM table_2_18444214_1 WHERE col4 = 'king's cup 2000';
Table: 2-18444214-1
7284
SELECT Tie no FROM 2-18711843-4 WHERE Home team = 'port vale';
Execute successfully ...
Execute successfully ...
7285
SELECT Score FROM 2-18711843-4 WHERE Home team = 'liverpool';
Execute successfully ...
Execute successfully ...
7286
SELECT Away team FROM 2-18711843-4 WHERE Tie no = 3;
Execute successfully ...
Execute successfully ...
7287
SELECT Murder (2012)

In [ ]:
# Log execution failures (first 5)
print(f"Number of Execution Failures: {len(failures)}")
if failures[:5]:
    print("Sample Execution Failures (Predicted, Ground Truth, Input):")
    for p, gt, inp in failures[:5]:
        print(f"Input: {inp}\nPred: {p}\nGT: {gt}\n")

Number of Execution Failures: 1150
Sample Execution Failures (Predicted, Ground Truth, Input):
Input: Question: What is the local name given to the capital of Anguilla? | Table: 1-1008653-1 (Country ( exonym ), Capital ( exonym ), Country ( endonym ), Capital ( endonym ), Official or native language(s) (alphabet/script))
Pred: SELECT Local ( exonym ) FROM 1-1008653-1 WHERE Capital ( endonym ) = 'Angulla';
GT: SELECT Capital ( endonym ) FROM 1-1008653-1 WHERE Country ( endonym ) = 'Anguilla';

Input: Question: What is the English name given to the city of St. John's? | Table: 1-1008653-1 (Country ( exonym ), Capital ( exonym ), Country ( endonym ), Capital ( endonym ), Official or native language(s) (alphabet/script))
Pred: SELECT Capital ( exonym ) FROM 1-1008653-1 WHERE Country ( endonym ) = 'St. John's';
GT: SELECT Capital ( exonym ) FROM 1-1008653-1 WHERE Capital ( endonym ) = 'St. John's';

Input: Question: What is the amount of trees, that require replacement when prevailing types

- Execution Accuracy: 0.6350

In [ ]:
# Evaluate 3: BLEU Score
from sacrebleu.metrics import BLEU

dev_output_texts = [item['output_text'] for item in dev_data_processed]

bleu = BLEU()
bleu_score = bleu.corpus_score(predictions, [dev_output_texts]).score / 100  # Normalize to [0, 1]
print(f"BLEU Score: {bleu_score:.4f}")

# Log sample low-BLEU predictions
bleu_scores = [bleu.sentence_score(pred, [gt]).score / 100 for pred, gt in zip(predictions, dev_output_texts)]
low_bleu = [(pred, gt, item['input_text'], score) for pred, gt, item, score in zip(predictions, dev_output_texts, dev_data_processed, bleu_scores) if score < 0.5]
print(f"Number of Low BLEU Scores (< 0.5): {len(low_bleu)}")
if low_bleu[:5]:
    print("Sample Low BLEU Predictions (Predicted, Ground Truth, Input, BLEU):")
    for p, gt, inp, score in low_bleu[:5]:
        print(f"Input: {inp}\nPred: {p}\nGT: {gt}\nBLEU: {score:.4f}\n")

BLEU Score: 0.8966


Streaming output truncated to the last 5000 lines.


Number of Low BLEU Scores (< 0.5): 125
Sample Low BLEU Predictions (Predicted, Ground Truth, Input, BLEU):
Input: Question: When did the Prime Minister of Italy take office? | Table: 1-10026563-1 (Entered office as Head of State or Government, Began time as senior G8 leader, Ended time as senior G8 leader, Person, Office)
Pred: SELECT Office FROM 1-10026563-1 WHERE Office = 'Premier Minister of Italy';
GT: SELECT Entered office as Head of State or Government FROM 1-10026563-1 WHERE Office = 'Prime Minister of Italy';
BLEU: 0.4432

Input: Question: How many teams were listed as runner up in 2005 and there the first semi finalist was Western Carolina? | Table: 1-11214772-2 (Year, Champion, Score, Runner-Up, Location, Semi-Finalist #1, Semi-Finalist #2)
Pred: SELECT COUNT(Semi-Finalist #2) FROM 1-11214772-2 WHERE Runner-Up = 2005 AND Location = 'West Carolina';
GT: SELECT COUNT(Runner-Up) FROM 1-11214772-2 WHERE Semi-Finalist #1 = 'Western Carolina' AND Year = 2005;
BLEU: 0.4363

Input: Q

- Bleu Score: 0.8966

In [ ]:
# Evaluate 4: ROUGE Scores
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
rouge_scores = [scorer.score(gt, pred) for gt, pred in zip(dev_output_texts, predictions)]

# Aggregate ROUGE scores
rouge1_f1 = sum(score['rouge1'].fmeasure for score in rouge_scores) / len(rouge_scores)
rouge2_f1 = sum(score['rouge2'].fmeasure for score in rouge_scores) / len(rouge_scores)
rougeL_f1 = sum(score['rougeL'].fmeasure for score in rouge_scores) / len(rouge_scores)
print(f"ROUGE-1 F1 Score: {rouge1_f1:.4f}")
print(f"ROUGE-2 F1 Score: {rouge2_f1:.4f}")
print(f"ROUGE-L F1 Score: {rougeL_f1:.4f}")

ROUGE-1 F1 Score: 0.9707
ROUGE-2 F1 Score: 0.9317
ROUGE-L F1 Score: 0.9572


- ROUGE-1 F1 Score: 0.9707
- ROUGE-2 F1 Score: 0.9317
- ROUGE-L F1 Score: 0.9572

In [ ]:
# Log sample low-ROUGE predictions (ROUGE-L < 0.5)
low_rouge = [(pred, gt, item['input_text'], score['rougeL'].fmeasure)
             for pred, gt, item, score in zip(predictions, dev_output_texts, dev_data_processed, rouge_scores)
             if score['rougeL'].fmeasure < 0.5]
print(f"Number of Low ROUGE-L Scores (< 0.5): {len(low_rouge)}")
if low_rouge[:5]:
    print("Sample Low ROUGE-L Predictions (Predicted, Ground Truth, Input, ROUGE-L):")
    for p, gt, inp, score in low_rouge[:5]:
        print(f"Input: {inp}\nPred: {p}\nGT: {gt}\nROUGE-L: {score:.4f}\n")

Number of Low ROUGE-L Scores (< 0.5): 2
Sample Low ROUGE-L Predictions (Predicted, Ground Truth, Input, ROUGE-L):
Input: Question: How many teams were listed as runner up in 2005 and there the first semi finalist was Western Carolina? | Table: 1-11214772-2 (Year, Champion, Score, Runner-Up, Location, Semi-Finalist #1, Semi-Finalist #2)
Pred: SELECT COUNT(Semi-Finalist #2) FROM 1-11214772-2 WHERE Runner-Up = 2005 AND Location = 'West Carolina';
GT: SELECT COUNT(Runner-Up) FROM 1-11214772-2 WHERE Semi-Finalist #1 = 'Western Carolina' AND Year = 2005;
ROUGE-L: 0.4706

Input: Question: Who is the company Investing Dragons, or tiny box? | Table: 2-12617978-10 (Episode, First aired, Entrepreneur(s), Company or product name, Money requested (£), Description of product, Investing Dragon(s))
Pred: SELECT Company or product name FROM 2-12617978-10 WHERE Investing Dragon(s) ='small box';
GT: SELECT Investing Dragon(s) FROM 2-12617978-10 WHERE Company or product name = 'tiny box';
ROUGE-L: 0.4667
